In [49]:
import requests
import json
import pandas as pd
from tqdm import tqdm

# Function to analyze movie sentiment
def analyze_movie_sentiment(movie_title, movie_description, model="deepseek-coder"):
    negative1 = "sad"
    positive1 = "happy"
    """Analyze sentiment for a movie using Ollama models with system/prompt separation"""
    
    # Split into system instructions and user prompt
    system = """
    You are a movie sentiment analyzer. When provided with a movie title and description please do the following in order:
    
    1. Evaluate the movie's script, main plot points, character dynamics, and character development. 
    2. Analyze the overall emotional tone and themes in the movie.
    3. Think about the sentiment in terms of a scale between the two sentiment words. ONLY think of sentiment in terms of these two words: {negative1}, {positive1}.
    4. Summarize your thoughts on the movie's sentiment in terms of the two sentiment words in less than 100 words
    5. Assign a sentiment score between -1.0 (very sad) and 1.0 (very happy) with at least 4 decimal places.
    
    Do not include any other text, preamble, or commentary outside of the prompted JSON structure.
    """
    
    # The actual prompt is just the movie information
    prompt = f"""Title: {movie_title}
    Description: {movie_description}
    Sentiment Words:{negative1}, {positive1}
    Evaluate this movie and assign the movie a sentiment score between -1.0 (incredibly {negative1}) and 1.0 (incredibly {positive1})
    Respond ONLY IMPORTANT: Respond ONLY with a valid JSON object with keys "score" and "explanation". 
    """
    
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "system": system,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload, stream=True)
        # Print raw response for debugging
        print(f"Raw response status: {response.status_code}")
        print(f"Raw response text: {response.text[:200]}...")  # Print first 100 chars
        
        # Try to parse the response as JSON
        try:
            response_data = response.json()
            print(f"Successfully parsed JSON response")
            print(response_data)
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            # Handle plain text response
            response_data = {"response": response.text}
            print(f"Treating as plain text response")
    
        # Extract the response text
        response_text = response_data.get("response", "")
        print(response_text)
        
        # Try to find and parse JSON in the response text
        sentiment_analysis = {"score": 0, "explanation": "Failed to parse response"}
        try:
            # Try to extract JSON object from text if present
            if "{" in response_text and "}" in response_text:
                print("it is present")
                start_idx = response_text.find("{")
                end_idx = response_text.rfind("}") + 1
                json_str = response_text[start_idx:end_idx]

                
                # Clean up potential issues in JSON string
                # Replace single quotes with double quotes if needed
                json_str = json_str.replace("'", '"')

                print("json_str")
                print(json_str)
                
                sentiment_analysis = json.loads(json_str)
                print(f"Successfully extracted JSON from response text")
                print(sentiment_analysis)
            else:
                print("it is not present")
                # If no JSON found, try to estimate sentiment from text
                print(f"No JSON found in response, estimating sentiment")
                if "positive" in response_text.lower():
                    score = 0.5
                elif "negative" in response_text.lower():
                    score = -0.5
                else:
                    score = 0
                
                sentiment_analysis = {
                    "score": score,
                    "explanation": response_text[:200]  # First 200 chars as explanation
                }
        except Exception as e:
            print(f"Error parsing sentiment: {e}")
            sentiment_analysis = {
                "score": 0,
                "explanation": f"Error parsing response: {str(e)[:100]}"
            }
    
    except Exception as e:
        print(f"Request error: {e}")
        sentiment_analysis = {
            "score": 0,
            "explanation": f"Error connecting to Ollama: {str(e)[:100]}"
        }
    
    # Return sentiment analysis results
    return {
        "title": movie_title,
        "sentiment_score": sentiment_analysis.get("score", 0),
        "explanation": sentiment_analysis.get("explanation", "")
    }


In [50]:
# Example usage
if __name__ == "__main__":
    # Sample movie data
    movies = [
        {"title": "The Shawshank Redemption", "description": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency."},
        ]
    movies = movies * 1
    
    # Choose model from Ollama (deepseek, mistral, etc.)
    model = "mistral"  # Change to your preferred model
    
    # Check if Ollama server is responding
    try:
        health_check = requests.get("http://localhost:11434/api/tags")
        print(f"Ollama server health check: {health_check.status_code}")
        if health_check.status_code == 200:
            print("Available models:")
            for model_info in health_check.json().get("models", []):
                print(f" - {model_info.get('name')}")
    except Exception as e:
        print(f"Warning: Couldn't connect to Ollama server: {e}")
        print("Make sure Ollama is running and listening on port 11434")

    results = []
    for movie in tqdm(movies):
        result = analyze_movie_sentiment(movie["title"], movie["description"], model)
        results.append(result)
    
    # Create a DataFrame for better visualization
    df = pd.DataFrame(results)
    print("\nResults:")
    print(df[["title", "sentiment_score", "explanation"]])
    
    # You can save the results for future use
    df.to_json("movie_sentiment_analysis.json", orient="records")

Ollama server health check: 200
Available models:
 - mistral:latest
 - deepseek-r1:1.5b
 - llama3.2:latest


100%|█████████████████████████████████████████████| 1/1 [00:31<00:00, 31.19s/it]

Raw response status: 200
Raw response text: {"model":"mistral","created_at":"2025-03-16T18:26:31.781072Z","response":" {\n      \"score\": \"0.5234\",\n      \"explanation\": \"The Shawshank Redemption exhibits strong character development and ...
Successfully parsed JSON response
{'model': 'mistral', 'created_at': '2025-03-16T18:26:31.781072Z', 'response': ' {\n      "score": "0.5234",\n      "explanation": "The Shawshank Redemption exhibits strong character development and deep, meaningful relationships, which counterbalance the predominantly sad theme of incarceration. The emotional tone swings between despair and hope, ultimately offering a sense of redemption and happiness that earns it a relatively positive sentiment score."\n   }', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 3055, 1763, 1228, 1032, 6762, 22558, 10916, 4274, 29491, 2452, 4625, 1163, 1032, 6762, 4709, 1072, 6204, 5433, 1279, 1040, 3064, 1065, 2513, 29515, 781, 1028, 781, 1028, 29508, 29491, 2499

                      title sentiment_score  \
0  The Shawshank Redemption          0.5234   

                                         explanation  
0  The Shawshank Redemption exhibits strong chara...  


In [46]:
response

NameError: name 'response' is not defined

In [30]:
response

NameError: name 'response' is not defined